##### Using Sparse Matrix to perform vectorsation

What is sparse matrix?
  - In a sparse matrix, most of the elements have a value of zero, which means that the matrix contains a lot of redundant information. Therefore, operations on sparse matrices can be significantly faster than on dense matrices because the computations can skip over the zero elements. Sparse matrices are often used in situations where memory usage is a concern, such as in large-scale scientific simulations, machine learning, and data processing.

How is it different from dense matrix?
  - Dense matrices have a high density of non-zero elements, and therefore, they contain a lot of useful information. Operations on dense matrices can be more computationally expensive than on sparse matrices because every element needs to be processed. Dense matrices are commonly used in situations where precision is critical, such as in numerical analysis and linear algebra.

What is the challenges in coding for sparse matrix?
  - Coding for sparse matrices can be more complex than coding for dense matrices due to the additional data structures and algorithms needed to handle the sparsity. 
  - Working with sparse matrices requires understanding the underlying data structures, such as Compressed Sparse Row (CSR) or Compressed Sparse Column (CSC) formats, and using specialized algorithms, such as sparse matrix multiplication or sparse matrix factorization. These additional complexities can make coding for sparse matrices more challenging.
  - In contrast, dense matrices can be more straightforward to code since they use standard data structures such as arrays and require standard algorithms such as matrix multiplication and inversion. But requires higher memory and computational resources.


Runtime results
  - For the same set of the data and data massaging performed.
  - Does not require to run in different batches (due to lesser memory and computational resources required), unlike in script done in _dense_matrix.ipynb
  - 12 mins for ~179k rows to match again 3.6k rows for 3 concurrent CPU workers

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
import pandas as pd
import re
from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


print('Just checking how many rows are there in the source excelfile...')
raw_df_vendor_customer = pd.read_excel('./Vendor Customer.xlsx')
raw_df_interested_party = pd.read_excel('./Interested Parties.xlsx')
print('Number of rows in raw_df_vendor_customer', len(raw_df_vendor_customer))
print('Number of rows in raw_df_interested_party', len(raw_df_interested_party))


print('ANALYSES BEGINS! :)')
# loading and reading into dataframe
df_vendor_customer = pd.read_excel('./Vendor Customer.xlsx')
df_interested_party = pd.read_excel('./Interested Parties.xlsx')
print('Complete: Data loaded into DF', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer', len(df_vendor_customer))
print('Number of rows in df_interested_party', len(df_interested_party))


print('Start: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# set the index as a column, to be used as a mapping field to join df_vendor_customer
df_interested_party = df_interested_party.reset_index().rename(columns={'index': 'index_ID'})

# to replace NULL/NaN values with empty strings
df_vendor_customer['Name'] = df_vendor_customer['Name'].fillna('')
df_interested_party['Interested Party List'] =df_interested_party['Interested Party List'].fillna('')


# define a regular expression that matches all non-alphanumeric and non-space characters and remove them
pattern = re.compile(r'[^\w\s]+')

df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name'].apply(lambda x: re.sub(pattern, '', x))
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List'].apply(lambda x: re.sub(pattern, '', x))


# update strings to all uppercase()
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.upper()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.upper()


# define the list of common words to remove, to remove noise (similar to stopwords concept)
# create a regular expression pattern that includes word boundaries (\b) before and after each word in the list of words to remove. This ensures that the str.replace method only removes the word when it appears as a standalone word, and not as a substring of other words.
words_to_remove = ['PTE', 'LTD', 'LLC', 'CO', 'SDN', 'BHD', 'PTY LIMITED', 'PTY', 'LIMITED', 'PVT', 'PRIVATE', 'INC', 'LLP', 'COMPANY']
pattern = r'\b(' + '|'.join(words_to_remove) + r')\b'


# for word in words_to_remove:
#     df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.replace(pattern, '', regex=True)
#     df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.replace(pattern, '', regex=True)


# update strings to remove leading and trailing whitespaces
df_vendor_customer['Name_Cleaned'] = df_vendor_customer['Name_Cleaned'].str.strip()
df_interested_party['Interested Party List_Cleaned'] = df_interested_party['Interested Party List_Cleaned'].str.strip()

# to drop duplicated rows
df_vendor_customer = df_vendor_customer.drop_duplicates()
df_interested_party = df_interested_party.drop_duplicates()

print('Complete: Data Massaging', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Number of rows in df_vendor_customer after data massaging', len(df_vendor_customer))
print('Number of rows in df_interested_party after data massaging', len(df_interested_party))


Just checking how many rows are there in the source excelfile...
Number of rows in raw_df_vendor_customer 186197
Number of rows in raw_df_interested_party 3587
ANALYSES BEGINS! :)
Complete: Data loaded into DF 2023-04-15 17:51:48
Number of rows in df_vendor_customer 186197
Number of rows in df_interested_party 3587
Start: Data Massaging 2023-04-15 17:51:48
Complete: Data Massaging 2023-04-15 17:51:49
Number of rows in df_vendor_customer after data massaging 176158
Number of rows in df_interested_party after data massaging 3587


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
from datetime import datetime
import scipy.sparse as sp
import joblib

print('Start: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# Define the TfidfVectorizer and fit it on the combined text column of both dataframes
vectorizer = TfidfVectorizer() # text extraction technique for text
vectorizer.fit(pd.concat([df_vendor_customer['Name_Cleaned'], df_interested_party['Interested Party List_Cleaned']]))

# Create a sparse matrix representation of the text column for each dataframe, TfidfVectorizer uses sparse matrix by default. To convert to dense matrix, use .toarray(). 
vendor_customer_matrix = vectorizer.transform(df_vendor_customer['Name_Cleaned'])
interested_party_matrix = vectorizer.transform(df_interested_party['Interested Party List_Cleaned'])
print('Vendor-Customer matrix shape:', vendor_customer_matrix.shape)
print('Interested Party matrix shape:', interested_party_matrix.shape)


""" Initialize the similarity matrix as a sparse matrix. 
# The .shape attribute is a property of a NumPy array or a sparse matrix in Python. It returns a tuple containing the dimensions of the array or matrix, in the format (number of rows, number of columns).
# .shape[0] represents to get the number of rows in both vendor_customer_matrix and interested_party_matrix. Please note the matrix is based on a single column, and therefore .shape[0] would make sense
# similarity matrix to have the same number of rows as the vendor_customer_matrix and the same number of columns as the interested_party_matrix.
"""
similarity_matrix = sp.lil_matrix((vendor_customer_matrix.shape[0], interested_party_matrix.shape[0]))
print('Similarity Matrix Shape:', similarity_matrix)

print('End: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))



def compute_cosine_similarity(vendor_customer_matrix, interested_party_matrix, n_jobs, batch_size):
    """Compute cosine similarity between two sparse matrices using Joblib's parallel processing.

    Args:
        vendor_customer_matrix (scipy.sparse.csr_matrix): The sparse matrix for the 'Name_Cleaned' column in df_vendor_customer.
        interested_party_matrix (scipy.sparse.csr_matrix): The sparse matrix for the 'Interested Party List_Cleaned' column in df_interested_party.
        n_jobs (int): The number of parallel jobs to run. Defaults to -1, which uses all available CPUs (not receommend, use -2 or -3 ideally)

    Returns:
        pandas.DataFrame: A dataframe containing the most similar row and its score for each row in vendor_customer_matrix.
    """
    # Normalize the rows of the two matrices
    norm_1 = np.sqrt(np.asarray(vendor_customer_matrix.power(2).sum(axis=1)).flatten())
    norm_2 = np.sqrt(np.asarray(interested_party_matrix.power(2).sum(axis=1)).flatten())

    norm_1[norm_1 == 0] = 1
    norm_2[norm_2 == 0] = 1

    normalized_matrix_1 = vendor_customer_matrix.multiply(1 / norm_1[:, np.newaxis])
    normalized_matrix_2 = interested_party_matrix.multiply(1 / norm_2[:, np.newaxis])

    # Compute the dot product between the rows of the two matrices
    def compute_dot_product(row_idx):
        
        # similarity_scores = normalized_matrix_1.getrow(row_idx).dot(normalized_matrix_2.T).toarray().flatten()
        # max_idx = np.argmax(similarity_scores)
        # return pd.Series({'most_similar_row': max_idx, 'similarity_score': similarity_scores[max_idx]})
        
        similarity_scores = normalized_matrix_1.getrow(row_idx).dot(normalized_matrix_2.T).toarray().flatten()
        
        max_idx = np.argmax(similarity_scores)
        most_similar_Interested_Party_List_Cleaned = df_interested_party.iloc[max_idx]['Interested Party List_Cleaned']
        similarity_score = similarity_scores[max_idx]
        row = df_vendor_customer.iloc[row_idx]
        
        return pd.Series({'Name_Cleaned': row['Name_Cleaned'], 'most_similar_Interested Party List_Cleaned': most_similar_Interested_Party_List_Cleaned, 'similarity_score': similarity_score})

    results = Parallel(n_jobs=n_jobs, verbose=10, batch_size=batch_size)(delayed(compute_dot_product)(row_idx) for row_idx in range(normalized_matrix_1.shape[0]))

    return pd.DataFrame(results)


# Define the batch size and number of jobs (ie CPU Cores) for parallel processing
n_jobs = -2
batch_size = 1000
print('Number of CPU cores available:', joblib.cpu_count(), '\nNumber of CPU cores to use:', n_jobs, '\nBatch Size:', batch_size)



print('Start: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# Call the function and store the resulting dataframe in a variable
result_df = compute_cosine_similarity(vendor_customer_matrix, interested_party_matrix, n_jobs, batch_size)
print('End: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


print('Start: Writing to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# result_df.head()
result_df.to_excel('results_sparse.xlsx')
print('End: Writing to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))



Start: Vectorisation 2023-04-15 17:51:49
End: Vectorisation 2023-04-15 17:51:52
Number of CPU cores available: 4 
Number of CPU cores to use: -2 
Batch Size: 1000
Vendor-Customer matrix shape: (176158, 70792)
Interested Party matrix shape: (3587, 70792)
Start: Parallel Processing 2023-04-15 17:51:52


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-2)]: Done 2006 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-2)]: Done 7006 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-2)]: Done 12006 tasks      | elapsed:   57.1s
[Parallel(n_jobs=-2)]: Done 19006 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-2)]: Done 26006 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-2)]: Done 35006 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-2)]: Done 44006 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-2)]: Done 55006 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-2)]: Done 66006 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-2)]: Done 79006 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-2)]: Done 92006 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-2)]: Done 107006 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-2)]: Done 122006 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-2)]: Done 13

End: Parallel Processing 2023-04-15 18:06:21
Start: Writing to Excel 2023-04-15 18:06:21
End: Writing to Excel 2023-04-15 18:06:37


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
from datetime import datetime
import scipy.sparse as sp
import joblib
from fuzzywuzzy import fuzz

print('Start: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# Define the TfidfVectorizer and fit it on the combined text column of both dataframes
vectorizer = TfidfVectorizer()
vectorizer.fit(pd.concat([df_vendor_customer['Name_Cleaned'], df_interested_party['Interested Party List_Cleaned']]))

# Create a sparse matrix representation of the text column for each dataframe
vendor_customer_matrix = vectorizer.transform(df_vendor_customer['Name_Cleaned'])
interested_party_matrix = vectorizer.transform(df_interested_party['Interested Party List_Cleaned'])

# Initialize the similarity matrix as a sparse matrix
similarity_matrix = sp.lil_matrix((vendor_customer_matrix.shape[0], interested_party_matrix.shape[0]))

print('End: Vectorisation', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

def compute_levenshtein_similarity(vendor_customer_matrix, interested_party_matrix, n_jobs, batch_size):
    """Compute Levenshtein similarity between two sparse matrices using Joblib's parallel processing.

    Args:
        vendor_customer_matrix (scipy.sparse.csr_matrix): The sparse matrix for the 'Name_Cleaned' column in df_vendor_customer.
        interested_party_matrix (scipy.sparse.csr_matrix): The sparse matrix for the 'Interested Party List_Cleaned' column in df_interested_party.
        n_jobs (int): The number of parallel jobs to run. Defaults to -1, which uses all available CPUs.

    Returns:
        pandas.DataFrame: A dataframe containing the most similar row and its score for each row in vendor_customer_matrix.
    """
    # Compute the Levenshtein distance between the rows of the two matrices
    def compute_levenshtein_distance(row_idx):
        
        similarity_scores = [fuzz.token_set_ratio(vendor_customer_matrix[row_idx], interested_party_matrix[idx]) for idx in range(interested_party_matrix.shape[0])]
        
        max_idx = np.argmax(similarity_scores)
        most_similar_Interested_Party_List_Cleaned = df_interested_party.iloc[max_idx]['Interested Party List_Cleaned']
        similarity_score = similarity_scores[max_idx]
        row = df_vendor_customer.iloc[row_idx]
        
        return pd.Series({'Name_Cleaned': row['Name_Cleaned'], 'most_similar_Interested Party List_Cleaned': most_similar_Interested_Party_List_Cleaned, 'similarity_score': similarity_score})

    results = Parallel(n_jobs=n_jobs, verbose=10, batch_size=batch_size)(delayed(compute_levenshtein_distance)(row_idx) for row_idx in range(vendor_customer_matrix.shape[0]))

    return pd.DataFrame(results)

# Define the batch size and number of jobs (ie CPU Cores) for parallel processing
n_jobs = -2
batch_size = 1000
print('Number of CPU cores available:', joblib.cpu_count(), '\nNumber of CPU cores to use:', n_jobs, '\nBatch Size:', batch_size)

print('Vendor-Customer matrix shape:', vendor_customer_matrix.shape)
print('Interested Party matrix shape:', interested_party_matrix.shape)

print('Start: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# Call the function and store the resulting dataframe in a variable
result_df = compute_levenshtein_similarity(vendor_customer_matrix, interested_party_matrix, n_jobs, batch_size)
print('End: Parallel Processing', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


print('Start: Writing to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
# result_df.head()
result_df.to_excel('results_sparse.xlsx')
print('End: Writing to Excel', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Start: Vectorisation 2023-04-15 18:09:50
End: Vectorisation 2023-04-15 18:09:52
Number of CPU cores available: 4 
Number of CPU cores to use: -2 
Batch Size: 1000
Vendor-Customer matrix shape: (176158, 70792)
Interested Party matrix shape: (3587, 70792)
Start: Parallel Processing 2023-04-15 18:09:52


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-2)]: Done 2006 tasks      | elapsed: 66.9min
[Parallel(n_jobs=-2)]: Done 7006 tasks      | elapsed: 117.9min
[Parallel(n_jobs=-2)]: Done 12006 tasks      | elapsed: 147.9min
[Parallel(n_jobs=-2)]: Done 19006 tasks      | elapsed: 232.7min
[Parallel(n_jobs=-2)]: Done 26006 tasks      | elapsed: 334.5min
[Parallel(n_jobs=-2)]: Done 35006 tasks      | elapsed: 426.7min
[Parallel(n_jobs=-2)]: Done 44006 tasks      | elapsed: 547.1min
[Parallel(n_jobs=-2)]: Done 55006 tasks      | elapsed: 645.3min
[Parallel(n_jobs=-2)]: Done 66006 tasks      | elapsed: 788.8min
